In [1]:
import tensorflow as tf

The basic principle of TensorFlow is simple: you first define in Python a graph of computations to perform (for example, the one in Figure 9-1), and then TensorFlow takes that graph and runs it efficiently using optimized C++ code.

<figure><img src='figure9-1.png'><figcaption>Figure 9-1</figcaption></figure>

## Create graph and run it in a session

In [2]:
# Construction phase: Setup the graph
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

# Execution phase:
# Create a session, initialize the variable, evaluate and close the session
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print('The Result is %d' %result)
sess.close()

# A better way is to write in block to make it cleaner
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print('The Result is %d' %result)

# We can also use init code to initialize all variable at once:
init = tf.global_variables_initializer()  # prepare an init node
with tf.Session() as sess:
    init.run()  # actually initialize all the variables
    result = f.eval()
print('The Result is %d' %result)

The Result is 42
The Result is 42
The Result is 42


## Manage Graphs
Any node you create is automatically added to the default graph. But if you wan to manage multiple independent graphs, we can create new graph add node inside a with block

In [3]:
x1 = tf.Variable(1)
print('Is x1 in default graph? %s' %str(x1.graph is tf.get_default_graph()))

graph = tf.Graph() # create a graph
with graph.as_default():
    x2 = tf.Variable(2)
print('Is x2 in default graph? %s' %str(x2.graph is tf.get_default_graph()))
print('Is x2 in graph? %s' %str(x2.graph is graph))

Is x1 in default graph? True
Is x2 in default graph? False
Is x2 in graph? True


We can also reset default graph

In [4]:
x1 = tf.Variable(1)
tf.reset_default_graph() # remove all the nodes
print('Is x1 in default graph? %s' %str(x1.graph is tf.get_default_graph()))

Is x1 in default graph? False


## Lifecycle of a Node Value
When evaluating a node, TensorFlow automatically determines the set of nodes that it depends on and it evaluates these nodes first. It **_will_** not reuse the result of the previous evaluation.

In [5]:
w = tf.constant(1)
x = w + 1
y = x + 1
z = x * 2

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

3
4


In the preceding code, w and x are evaluated twice. If we want to be more efficient, we can evaluate them in one graph run

In [6]:
with tf.Session() as sess:
    y_eval, z_eval = sess.run([y, z])
    print(y_eval)
    print(z_eval)

3
4


## Linear Regression with TensorFlow
TensorFlow operations can take any number of inputs and produce any number of outputs. The inputs and ouputs are multidimensional arrays, called *tensor*. Tensors have a type and a shape. In Python API tensors are simply represented by NumPy ndarrays. 

In [7]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    

## Implementing Gradient Descent
* TensorFlow provides a number of optimizers out of the box, including a Gradient Descent optimizer

In [8]:
from sklearn.preprocessing import MinMaxScaler

n_epochs = 1000
learning_rate = 0.01

housing = fetch_california_housing()
m, n = housing.data.shape
scaler = MinMaxScaler()
scalre_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scalre_housing_data]
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch ', epoch, 'MSE = ', mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch  0 MSE =  1.3835073
Epoch  100 MSE =  1.3274424
Epoch  200 MSE =  1.279127
Epoch  300 MSE =  1.2346176
Epoch  400 MSE =  1.1934859
Epoch  500 MSE =  1.1553848
Epoch  600 MSE =  1.1200206
Epoch  700 MSE =  1.0871432
Epoch  800 MSE =  1.0565367
Epoch  900 MSE =  1.028013


## Mini-batch Gradient Descent

We can feed data to evaluation

In [9]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A: [[1,2,3],[2,3,4]]})
    
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[6. 7. 8.]
 [7. 8. 9.]]


In [10]:
def fetch_batch(epoch, batch_index, batch_size):
    
    batch_start = batch_index*batch_size
    batch_end = batch_start + batch_size
    X_batch = scaled_housing_data_plus_bias[batch_start:batch_end, :]
    y_batch = housing.target.reshape(-1, 1)[batch_start:batch_end, :]
    assert X_batch.shape[0] == y_batch.shape[0]
    return X_batch, y_batch

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

init = tf.global_variables_initializer()


n_epochs = 1000
learning_rate = 0.01
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        if epoch % 100 == 0:
            print('Epoch ', epoch, 'MSE = ', mse.eval(feed_dict={X: X_batch, y: y_batch}))
            
    best_theta = theta.eval()

Epoch  0 MSE =  0.9356201
Epoch  100 MSE =  0.1406384
Epoch  200 MSE =  0.12077526
Epoch  300 MSE =  0.11835499
Epoch  400 MSE =  0.11879311
Epoch  500 MSE =  0.11948355
Epoch  600 MSE =  0.1200376
Epoch  700 MSE =  0.120447256
Epoch  800 MSE =  0.12075247
Epoch  900 MSE =  0.12098324


## Save and restore models
Once we have the model, we should save its parameters to disk so we can come back and use it. Moreover, we may want to save checkpoints at regular intervals during training in case computer crashes during training.

In [11]:
# Code from examples above

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 100
learning_rate = 0.01

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        if epoch % 100 == 0:
            print('Epoch ', epoch, 'MSE = ', mse.eval(feed_dict={X: X_batch, y: y_batch}))
            save_path = saver.save(sess, 'tmp/model.ckpt')
            
    best_theta = theta.eval()
    save_path = saver.save(sess, 'tmp/model_final.ckpt')

Epoch  0 MSE =  1.1665298


In [12]:
# Restore a saved model
saver = tf.train.Saver() # create a Saver at construction phase

with tf.Session() as sess:
    saver.restore(sess, 'tmp/model_final.ckpt') # instead of init node, call restore() method of the Saver object
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        if epoch %100 == 0:
            print('Epoch ', epoch, 'MSE = ', mse.eval(feed_dict={X: X_batch, y: y_batch}))

    best_theta = theta.eval()
            

INFO:tensorflow:Restoring parameters from tmp/model_final.ckpt
Epoch  0 MSE =  0.13864568


## Visualize the Graph


In [26]:
from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir = '{}/run-{}/'.format(root_logdir, now)

n_epoches = 100
n_batches = 100

X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y - y_pred
mse = tf.reduce_mean(tf.square(error), name='MSE')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

## create a node in graph that will evaluate MSE and write
## it to TensorBloar-compatible log string called summary
mse_summary = tf.summary.scalar('MSE', mse)
## write summaries to logfiles in log directory
## it will create directory if not exist
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
                
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()       

file_writer.close()

After create the logs, we can see it from TensorBoard server. Inside terminal for mac of command on windows:
``` 
tensorboard --logdir tf_logs/
```
You will see 
``` 
TensorBoard 1.6.0 at http:/0.0.0.0:6006 (Press CTRL+C to quit)
```
<figure><img src='figure9-2.png'><figcaption>Figure 9-2</figcaption></figure>
